In [1]:
# 🔧 STEP 1: Install required libraries
!pip install qdrant-client sentence-transformers google-generativeai --quiet
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

QDRANT_API_KEY = "QDRANT_API_KEY"  # 🔒 Paste your API key from Qdrant dashboard
QDRANT_URL = "QDRANT_URL"

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)


In [2]:
from sentence_transformers import SentenceTransformer

print("Loading embedding model (required for chat)...")
# This line loads the model and assigns it to the 'embedder' variable
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded.")

Loading embedding model (required for chat)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding model loaded.


In [ ]:
# 💬 STEP 9: Interactive Chat Loop

import requests
import json
from tqdm import tqdm # Can remove if you don't want per-hit progress during context building

# --- Ensure previous initializations are done ---
# Make sure 'client', 'embedder', 'LLM_URL', 'API_ENDPOINT', 'headers' are defined.
# Define MODEL_ENDPOINT_URL if not already done
if 'API_ENDPOINT' not in locals(): API_ENDPOINT = "/predict" # Set default if needed
if 'LLM_URL' not in locals(): LLM_URL = "https://11fc-34-126-176-47.ngrok-free.app" # Replace if needed
MODEL_ENDPOINT_URL = LLM_URL.strip('/') + "/" + API_ENDPOINT.strip('/')
print('MODEL_ENDPOINT_URL: ',MODEL_ENDPOINT_URL)
# --- Configuration for the Chat ---
SEARCH_LIMIT = 5 # How many results to fetch from Qdrant (adjust as needed)
REQUEST_TIMEOUT = 90 # Seconds to wait for LLM response

headers = {
    "Content-Type": "application/json"
    # Add "Authorization": "Bearer YOUR_KEY" if your endpoint needs it
}



while True:
    user_query = input("\nYou: ")
    if not user_query or user_query.lower() in ['quit', 'exit']:
        print("Exiting chat. Goodbye!")
        break

    # --- Stage 1: Retrieval (Adapted from Cell 7) ---
    print("Searching knowledge base...")
    results = []
    top_k_context = "No context retrieved." # Default
    try:
        query_vec = embedder.encode(user_query).tolist()
        # Use query_points (recommended) or search
        results = client.search( # Replace with client.query_points if preferred
            collection_name="alumni_profiles",
            query_vector=query_vec,
            limit=SEARCH_LIMIT
        )
        print(f"Retrieved {len(results)} potential profiles.")
    except Exception as e:
        print(f"Error during Qdrant search: {e}")
        continue # Skip to next loop iteration

    # --- Stage 2: Context Building (Adapted from Cell 7 - Comprehensive) ---
    print("Building context...")
    context_parts = []
    if results:
        # Using tqdm here might be noisy in chat, consider removing loop prints/tqdm
        for i, hit in enumerate(results): # tqdm(results, desc="Processing hits", leave=False):
            if not hit.payload: continue

            data = hit.payload
            name = data.get('Name', 'N/A')
            location = data.get('Location', '')
            work_exp_list = data.get('WorkExperience', [])
            education_list = data.get('Education', [])
            skills_list = data.get('Skills', [])
            email = data.get('Email', '')
            phone = data.get('Phone', '')

            if not isinstance(work_exp_list, list): work_exp_list = []
            if not isinstance(education_list, list): education_list = []
            if not isinstance(skills_list, list): skills_list = []
            safe_work_exp = [str(item) for item in work_exp_list]
            safe_education = [str(item) for item in education_list]
            safe_skills = [str(item) for item in skills_list]

            context_part = f"""Profile Entry {i+1}:
            Name: {str(name)}
            Location: {str(location)}
            Work Experience: {'; '.join(safe_work_exp)}
            Education: {'; '.join(safe_education)}
            Skills: {', '.join(safe_skills)}
            Email: {str(email)}
            Phone: {str(phone)}
            Search Score: {hit.score:.4f}
            """
            context_parts.append(context_part.strip())

        top_k_context = "\n\n=====\n\n".join(context_parts)
        print(f"Context built (Length: {len(top_k_context)} chars).")
    else:
        print("No relevant profiles found in knowledge base.")
        # Optional: Still send to LLM without context, or just reply directly
        # print("Assistant: I couldn't find any relevant information in the profiles.")
        # continue

    # --- Stage 3: Generation (Adapted from Cell 8 - Custom LLM) ---
    print("Generating response...")
    final_prompt = f"""
Use the following context to answer the user's question accurately based only on the provided text. If the answer is not found in the context, say so clearly.

Context:
{top_k_context}

Question: {user_query}
Answer:"""

    payload = {"text": final_prompt}
    generated_text = "Sorry, I encountered an error." # Default error

    try:
        response = requests.post(MODEL_ENDPOINT_URL, headers=headers, json=payload, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        response_data = response.json()
        try:
            generated_text = response_data["response"]
        except KeyError:
            generated_text = f"Error: 'response' key missing in LLM output. Raw: {str(response_data)[:200]}"
        except Exception as e:
             generated_text = f"Error parsing LLM response content: {e}"

    except requests.exceptions.Timeout:
        generated_text = "Sorry, the request to the language model timed out."
    except requests.exceptions.RequestException as e:
        generated_text = f"Sorry, error communicating with the language model: {e}"
    except Exception as e:
        generated_text = f"Sorry, an unexpected error occurred: {e}"

    print(f"\nAssistant: {generated_text}")

# Suggested example questions
suggestions = [
    "What is Virginia Hammond's contact information?",
    "Find alumni with Python programming skills.",
    "Find alumni who live in California.",
    "Find alumni who live in California and have MSc in Computer Science education",
    "Who completed their degree after 2018 and have MSc in Computer Science ?",
    "List alumni who majored in Electrical Engineering.",
    "who graduated with computer science degree around 2020 and works in data science domain?",
    "Who has experience with cloud computing?",
    "Who graduated between 2018 and 2021 ? share their degree major ?",
    "can you give information about John carter ?"
]
# Launch Gradio App
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎓 Alumni Information Chatbot")
    gr.Markdown("Ask questions about alumni profiles. Examples: `What are Virginia Hammond’s skills?`, `Find alumni who graduated after 2020 and live in California.`")

    chatbot = gr.Chatbot(label="Chat History")

    with gr.Row():
        txt = gr.Textbox(placeholder="Enter your question here...", label="Your Question", scale=9)
        send_btn = gr.Button("Send", scale=1)
        clear_btn = gr.Button("🧹 Clear Chat")

    gr.Markdown("**⚡ Suggested Questions (Click any example to try):**")

    with gr.Row():
        for q in suggestions[:5]:
            gr.Button(q).click(fn=lambda x=q: x, outputs=txt)
    with gr.Row():
        for q in suggestions[5:]:
            gr.Button(q).click(fn=lambda x=q: x, outputs=txt)

    def clear():
        return [], ""

    send_btn.click(chat_bot_fn, inputs=[txt, chatbot], outputs=[chatbot, txt])
    txt.submit(chat_bot_fn, inputs=[txt, chatbot], outputs=[chatbot, txt])
    clear_btn.click(clear, outputs=[chatbot, txt])

demo.launch(share=True)

MODEL_ENDPOINT_URL:  https://11fc-34-126-176-47.ngrok-free.app/predict
--- Alumni Profile Chat Bot ---
Ask questions about the alumni profiles. Type 'quit' or 'exit' to end.

You: What is the phone number of NAVARRO?
Searching knowledge base...


<ipython-input-4-08eef9d5e551>:40: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search( # Replace with client.query_points if preferred


Retrieved 5 potential profiles.
Building context...
Context built (Length: 3310 chars).
Generating response...

Assistant: +18034211100 
